In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [20]:
True_news = pd.read_csv('E:\Major Project\Project\True.csv')
Fake_news = pd.read_csv('E:\Major Project\Project\Fake.csv')

In [21]:
True_news['label'] = 0
Fake_news['label'] = 1

In [22]:
True_news.head()

,title,text,subject,date,label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",0
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",0
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",0
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",0
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",0


In [23]:
Fake_news.head()

,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",1
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",1
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",1
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",1
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",1


In [24]:
dataset1 = True_news[['text','label']]
dataset2 = Fake_news[['text','label']]

In [25]:
dataset = pd.concat([dataset1 , dataset2])

In [26]:
dataset.shape

(44898, 2)

In [27]:
dataset.isnull().sum() # no null values

text     0
label    0
dtype: int64

In [28]:
dataset['label'].value_counts()  # balanced or unbalanced dataset

1    23481
0    21417
Name: label, dtype: int64

In [29]:
dataset1.shape # true news

(21417, 2)

In [30]:
dataset2.shape # fake news

(23481, 2)

In [31]:
dataset = dataset.sample(frac = 1) # reshuffle/resample dataset

In [32]:
dataset.head(20)

,text,label
19113,The owner of a masonry business in Colorado is...,1
7316,The Washington Post reports that the Republica...,1
2725,(Reuters) - New York state Attorney General Er...,0
12200,BRUSSELS (Reuters) - The Belgian trial of one ...,0
3181,Donald Trump is a madman putting our nation on...,1
5053,WASHINGTON (Reuters) - Two senior senators ask...,0
15021,BUDAPEST (Reuters) - Hungary s right-wing Prim...,0
15650,SPOT ON RACHEL ZONATION The Obamas pride thems...,1
17458,"AIN ISSA, Syria/BEIRUT (Reuters) - Syrian Isla...",0
992,Protests against Trumpcare are ramping up now ...,1


In [33]:
import nltk

In [34]:
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [35]:
ps = WordNetLemmatizer()

In [36]:
stopwords = stopwords.words('english')

In [37]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\arinm\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [38]:
def cleaning_data(row):
    
    # convert text to into lower case
    row = row.lower() 
    
    # this line of code only take words from text and remove number and special character using RegX
    row = re.sub('[^a-zA-Z]' , ' ' , row)
    
    # split the data and make token.
    token = row.split() 
    
    # lemmatize the word and remove stop words like a, an , the , is ,are ...
    news = [ps.lemmatize(word) for word in token if not word in stopwords]  
    
    # finaly join all the token with space
    cleanned_news = ' '.join(news) 
    
    # return cleanned data
    return cleanned_news 

In [39]:
dataset['text'] = dataset['text'].apply(lambda x : cleaning_data(x))

In [40]:
dataset.isnull().sum()

text     0
label    0
dtype: int64

In [41]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [42]:
vectorizer = TfidfVectorizer(max_features = 50000 , lowercase=False , ngram_range=(1,2))

In [43]:
dataset.shape

(44898, 2)

In [44]:
X = dataset.iloc[:35000,0]
y = dataset.iloc[:35000,1]

In [45]:
X.head()

19113    owner masonry business colorado speaking decis...
7316     washington post report republican billionaire ...
2725     reuters new york state attorney general eric s...
12200    brussels reuters belgian trial one suspect beh...
3181     donald trump madman putting nation brink world...
Name: text, dtype: object

In [46]:
y.head()

19113    1
7316     1
2725     0
12200    0
3181     1
Name: label, dtype: int64

In [47]:
from sklearn.model_selection import train_test_split
train_data , test_data , train_label , test_label = train_test_split(X , y , test_size = 0.2 ,random_state = 0)

In [48]:
vec_train_data = vectorizer.fit_transform(train_data)

In [49]:
vec_train_data = vec_train_data.toarray()

In [50]:
train_data.shape , test_data.shape

((28000,), (7000,))

In [51]:
vec_test_data = vectorizer.transform(test_data).toarray()

In [52]:
vec_train_data.shape , vec_test_data.shape

((28000, 50000), (7000, 50000))

In [53]:
train_label.value_counts() # balanced partition

1    14546
0    13454
Name: label, dtype: int64

In [54]:
test_label.value_counts() # balanced partition

1    3701
0    3299
Name: label, dtype: int64

In [55]:
training_data = pd.DataFrame(vec_train_data , columns=vectorizer.get_feature_names())
testing_data = pd.DataFrame(vec_test_data , columns= vectorizer.get_feature_names())

In [56]:
from sklearn.naive_bayes import MultinomialNB

In [57]:
from sklearn.metrics import accuracy_score,classification_report

In [58]:
clf = MultinomialNB()

In [59]:
clf.fit(training_data, train_label)
y_pred  = clf.predict(testing_data)

# MULTINOMIAL NB

In [62]:
pd.Series(y_pred).value_counts()

1    3714
0    3286
dtype: int64

In [63]:
test_label.value_counts()

1    3701
0    3299
Name: label, dtype: int64

In [64]:
print(classification_report(test_label , y_pred))

              precision    recall  f1-score   support

           0       0.95      0.95      0.95      3299
           1       0.95      0.96      0.95      3701

    accuracy                           0.95      7000
   macro avg       0.95      0.95      0.95      7000
weighted avg       0.95      0.95      0.95      7000



In [65]:
y_pred_train = clf.predict(training_data)
print(classification_report(train_label , y_pred_train))

              precision    recall  f1-score   support

           0       0.96      0.96      0.96     13454
           1       0.96      0.96      0.96     14546

    accuracy                           0.96     28000
   macro avg       0.96      0.96      0.96     28000
weighted avg       0.96      0.96      0.96     28000



In [66]:
accuracy_score(train_label , y_pred_train)

0.9595714285714285

In [67]:
accuracy_score(test_label , y_pred)

0.9512857142857143

In [68]:
news = cleaning_data(str("Imposters posing as army personnel on the social media have been called out by the Indian Army as false news and disinformation."))

In [69]:
single_prediction = clf.predict(vectorizer.transform([news]).toarray())
single_prediction

array([0], dtype=int64)

# Model

In [71]:
import joblib 

In [72]:
joblib.dump(clf , 'model.pkl')

['model.pkl']

In [73]:
model = joblib.load('model.pkl')